In [1]:
import copy
import pyspark.sql.functions as f

In [2]:
workspace = 'dev'
apiVersion = "5.2"
moduleName = "Ed-Fi"
schoolYear = None
districtId = None
#metadataUrl = "https://github.com/ayunav/OpenEduAnalytics/raw/main/modules/module_catalog/Ed-Fi/utils/Metadata.csv"

swagger_url = "https://api.edgraph.com/edfi/v5.2/saas/tx/api/metadata/data/v3/edfi-swagger.json"
metadataUrl = "https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Ed-Fi/utils/Metadata.csv"

In [ ]:
#%run dev_v0_1_OEA_py

In [3]:
%run OEA_py

In [4]:
# 1) set the workspace (this determines where in the data lake you'll be writing to and reading from).
# You can work in 'dev', 'prod', or a sandbox with any name you choose.
# For example, Sam the developer can create a 'sam' workspace and expect to find his datasets in the data lake under oea/sandboxes/sam
workspace = 'dev'
oea.set_workspace(workspace)

In [5]:
schema_gen = OpenAPIUtil(swagger_url)
schemas = schema_gen.create_spark_schemas()
primitive_datatypes = ['timestamp', 'date', 'decimal', 'boolean', 'integer', 'string', 'long']

In [6]:
def get_descriptor_schema(descriptor):
    fields = []
    fields.append(StructField('_etag',LongType(), True))
    fields.append(StructField(f"{descriptor[:-1]}Id", IntegerType(), True))
    fields.append(StructField('codeValue',StringType(), True))
    fields.append(StructField('description',StringType(), True))
    fields.append(StructField('id',StringType(), True))
    fields.append(StructField('namespace',StringType(), True))
    fields.append(StructField('shortDescription',StringType(), True))
    return StructType(fields)

def get_descriptor_metadata(descriptor):
    return [['_etag', 'long', 'no-op'],
            [f"{descriptor[:-1]}Id", 'integer', 'hash'],
            ['codeValue','string', 'no-op'],
            ['description','string', 'no-op'],
            ['id','string', 'no-op'],
            ['namespace','string', 'no-op'],
            ['shortDescription','string', 'no-op']]

In [7]:
def has_column(df, col):
    try:
        df[col]
        return True
    except AnalysisException:
        return False

def modify_descriptor_value(df, col_name):
    if col_name in df.columns:
        # TODO: @Abhinav (Edit: Now @Viraj), I do not see where you made the changes to use the descriptorId instead of Namespace/CodeValue
        df = df.withColumn(f"{col_name}LakeId", f.concat_ws('_', f.col('DistrictId'), f.col('SchoolYear'), f.regexp_replace(col_name, '#', '_')))
        df = df.drop(col_name)
    else:
        df = df.withColumn(f"{col_name}LakeId", f.lit(None).cast("String"))

    return df

def flatten_reference_col(df, target_col):
    col_prefix = target_col.name.replace('Reference', '')
    df = df.withColumn(f"{col_prefix}LakeId", f.when(f.col(target_col.name).isNotNull(), f.concat_ws('_', f.col('DistrictId'), f.col('SchoolYear'), f.split(f.col(f'{target_col.name}.link.href'), '/').getItem(3))))
    df = df.drop(target_col.name)
    return df

def modify_references_and_descriptors(df, target_col):
    for ref_col in [x for x in df.columns if re.search('Reference$', x) is not None]:
        df = flatten_reference_col(df, target_col.dataType.elementType[ref_col])
    for desc_col in [x for x in df.columns if re.search('Descriptor$', x) is not None]:
        df = modify_descriptor_value(df, desc_col)
    return df

def explode_arrays(df, sink_general_path,target_col, schema_name, table_name):
    cols = ['lakeId', 'DistrictId', 'SchoolYear']
    child_df = df.select(cols + [target_col.name])
    child_df = child_df.withColumn("exploded", f.explode(target_col.name)).drop(target_col.name).select(cols + ['exploded.*'])

    # TODO: It looks like te {target_col.name}LakeId column is not addedd to the child entities
    #       We should use LakeId suffix when using the "id" column from the parent and HKey suffix when creating a Hash Key based on composite key columns
    identity_cols = [x.name for x in target_col.dataType.elementType.fields if 'x-Ed-Fi-isIdentity' in x.metadata].sort()
    if(identity_cols is not None and len(identity_cols) > 0):
        child_df = child_df.withColumn(f"{target_col.name}LakeId", f.concat(f.col('DistrictId'), f.lit('_'), f.col('SchoolYear'), f.lit('_'), *[f.concat(f.col(x), f.lit('_')) for x in identity_cols]))
    
    # IMPORTANT: We must modify Reference and Descriptor columns for child columns "first". 
    # This must be done "after" the composite key from identity_cols has been created otherwise the columns are renamed and will not be found by identity_cols.
    # This must be done "before" the grand_child is exploded below
    child_df = modify_references_and_descriptors(child_df, target_col)

    for array_sub_col in [x for x in target_col.dataType.elementType.fields if x.dataType.typeName() == 'array' ]:
        grand_child_df = child_df.withColumn('exploded', f.explode(array_sub_col.name)).select(child_df.columns + ['exploded.*']).drop(array_sub_col.name)
        
        # Modifying Reference and Descriptor columns for the grand_child array
        grand_child_df = modify_references_and_descriptors(grand_child_df, array_sub_col)

        # TODO: Pseudonimize and Write to Sensitive folder for child arrays?
        logger.info(f"Writing Grand Child Table - {table_name}_{target_col.name}_{array_sub_col.name}")
        oea.upsert(grand_child_df, 
                   f"{sink_general_path}_{target_col.name}_{array_sub_col.name}", 
                   'lakeId') 
        oea.add_to_lake_db(f"{sink_general_path}_{target_col.name}_{array_sub_col.name}")
        #grand_child_df.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(oea.to_url(f"{sink_general_path}_{target_col.name}_{array_sub_col.name}"))

    # TODO: Pseudonimize and Write to Sensitive folder for child arrays?
    logger.info(f"Writing Child Table - {table_name}_{target_col.name}")
    oea.upsert(child_df, 
               f"{sink_general_path}_{target_col.name}", 
              'lakeId') 
    oea.add_to_lake_db(f"{sink_general_path}_{target_col.name}")
    #child_df.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(oea.to_url(f"{sink_general_path}_{target_col.name}"))

    # Drop array column from parent entity
    df = df.drop(target_col.name)
    return df

def transform(df, 
              schema_name, 
              table_name, 
              primary_key,
              ext_entity,
              sink_general_path,
              parent_schema_name, 
              parent_table_name):
    # TODO: Pseudominization Pending
    if re.search('Descriptors$', table_name) is None:
        # Use Deep Copy otherwise the schemas object also gets modified every time target_schema is modified
        target_schema = copy.deepcopy(schemas[table_name])
        # Add primary key
        if has_column(df, primary_key):
            df = df.withColumn('lakeId', f.concat_ws('_', f.col('DistrictId'), f.col('SchoolYear'), f.col(primary_key)).cast("String"))
        else:
            df = df.withColumn('lakeId', f.lit(None).cast("String"))
    else:
        target_schema = get_descriptor_schema(table_name)
        # Add primary key
        if has_column(df, primary_key):
            # TODO: @Abhinav (Edit: Now @Viraj), I do not see where you made the changes to use the descriptorId instead of Namespace/CodeValue
            df = df.withColumn('lakeId', f.concat_ws('_', f.col('DistrictId'), f.col('SchoolYear'), f.col('namespace'), f.col('codeValue')).cast("String"))
        else:
            df = df.withColumn('lakeId', f.lit(None).cast("String"))

    target_schema = target_schema.add(StructField('DistrictId', StringType()))\
                                 .add(StructField('SchoolYear', StringType()))\
                                 .add(StructField('LastModifiedDate', TimestampType()))

    df = transform_sub_module(df, target_schema, sink_general_path, schema_name, table_name)
    logger.info(f"Writing Main Table - {table_name}")
    oea.upsert(df, 
               f"{sink_general_path}", 
              'lakeId') 
    #df.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(oea.to_url(sink_general_path))
    oea.add_to_lake_db(sink_general_path)

    if '_ext' in df.columns:
        target_schema = get_ext_entities_schemas(table_name = table_name,
                                                 ext_column_name = '_ext',
                                                 default_value = ext_entity)
        df = flatten_ext_column(df = df, 
                                table_name = table_name, 
                                ext_col = '_ext', 
                                inner_key = ext_entity,
                                ext_inner_cols = target_schema.fieldNames())
        sink_general_path = sink_general_path.replace('/ed-fi/', f'/{ext_entity.lower()}/')
        df = transform_sub_module(df, 
                                  target_schema, 
                                  sink_general_path, 
                                  schema_name,
                                  table_name)

        logger.info(f"Writing EXT Table - {table_name}")
        oea.upsert(df, 
               f"{sink_general_path}", 
              'lakeId') 
        #df.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(oea.to_url(sink_general_path))
        oea.add_to_lake_db(sink_general_path)
        

#df = spark.read.format('delta').load(f"{stage2_ingested}/ed-fi/absenceEventCategoryDescriptors")
#df = transform(df, "ed-fi", "absenceEventCategoryDescriptors", None, None)

def transform_sub_module(df, target_schema, sink_general_path, schema_name, table_name):
    for col_name in target_schema.fieldNames():
        target_col = target_schema[col_name]
        # If Primitive datatype, i.e String, Bool, Integer, etc.abs
        # Note: Descriptor is a String therefore is a Primitive datatype
        if target_col.dataType.typeName() in primitive_datatypes:
            # If it is a Descriptor
            if re.search('Descriptor$', col_name) is not None:
                df = modify_descriptor_value(df, col_name)
            else:
                if col_name in df.columns:
                    # Casting columns to primitive data types
                    df = df.withColumn(col_name, f.col(col_name).cast(target_col.dataType))
                else:
                    # If Column not present in dataframe, add column with None values.
                    df = df.withColumn(col_name, f.lit(None).cast(target_col.dataType))
        # If Complex datatype, i.e. Object, Array
        else:
            if col_name not in df.columns:
                df = df.withColumn(col_name, f.lit(None).cast(target_col.dataType))
            else:
                # Generate JSON column as a Complex Type
                df = df.withColumn(f"{col_name}_json", f.to_json(f.col(col_name))) \
                    .withColumn(col_name, f.from_json(f.col(f"{col_name}_json"), target_col.dataType)) \
                    .drop(f"{col_name}_json")
            
            # Modify the links with surrogate keys
            if re.search('Reference$', col_name) is not None:
                df = flatten_reference_col(df, target_col)
    
            if target_col.dataType.typeName() == 'array':
                df = explode_arrays(df, sink_general_path,target_col, schema_name, table_name)
    return df

In [8]:
def get_ext_entities_schemas(table_name = 'staffs',
                             ext_column_name = '_ext',
                             default_value = 'TPDM'):
    target_schema = copy.deepcopy(schemas[table_name])
    for col_name in target_schema.fieldNames():
        target_col = target_schema[col_name]
        if target_col.name == ext_column_name:
            if target_col.dataType[0].name == default_value:
                return target_col.dataType[0].dataType         
                
def flatten_ext_column(df, 
                       table_name, 
                       ext_col, 
                       inner_key,
                       ext_inner_cols
                       ):
    #TODO: Modify the complex sub type field name logic
    cols = ['lakeId', 'DistrictId', 'SchoolYear', 'id_pseudonym']
    flattened_cols = ext_inner_cols#["educatorPreparationPrograms"] #_ext_TX_cols[table_name]
    dict_col = F.col(ext_col)[inner_key]
    complex_dtype_text = str(df.select('_ext').dtypes[0][1])

    exprs = [dict_col.getItem(key).alias(key) for key in flattened_cols if str(key) in complex_dtype_text]
    flattened_df = df.select(exprs + cols)
    return flattened_df

In [14]:
def refine_and_explode_data(schema_name, 
                            tables_source,
                            ext_entity, 
                            transform_mode, 
                            test_mode,
                            test_items = []):
    global districtPath,schoolYearPath
    items = oea.get_folders(f"stage2/Ingested/{tables_source}")
    if test_mode:
        if test_items != 'All':
            items = test_items
    for item in items:
            table_name = item #sap_to_edfi_complex[item]
            table_path = f"{tables_source}/{item}"
            
            logger.info(f"Processing schema/table: {schema_name}/{table_name}")
            if item == 'metadata.csv':
                logger.info('ignore metadata processing, since this is not a table to be ingested')
            else:
                try:
                    if not(transform_mode):
                        df = oea.refine(table_path, 
                                        metadata = metadata[item], 
                                        primary_key = 'id')
                    if transform_mode:
                        logger.info('Ed-Fi to Ed-Fi Relationship Model: ' + table_name)               
                        source_path = f'stage2/Ingested/{table_path}'
                        sink_general_path, sink_sensitive_path = oea.get_sink_general_sensitive_paths(source_path)

                        df_changes = oea.get_latest_changes(source_path, sink_general_path)
                        df_changes = df_changes.withColumn('DistrictId', F.lit(districtPath))
                        df_changes = df_changes.withColumn('SchoolYear', F.lit(schoolYearPath))
                        
                        if df_changes.count() > 0:
                            df_pseudo, df_lookup = oea.pseudonymize(df_changes, 
                                                                    metadata, 
                                                                    transform_mode = True, 
                                                                    primary_key = 'id')
                            
                            transform(df_pseudo, schema_name, table_name, 
                                      'id_pseudonym', ext_entity, sink_general_path,
                                      None, None)
                            
                            oea.upsert(df_lookup, sink_sensitive_path, 'id')    
                            oea.add_to_lake_db(sink_sensitive_path)
                        else:
                            logger.info(f'No updated rows in {source_path} to process.')

                except AnalysisException as e:
                    # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
                    logger.info(e)

In [15]:
metadata = oea.get_metadata_from_url(metadataUrl)
districtPath = districtId if districtId != None else "All"
schoolYearPath = schoolYear if schoolYear != None else "All"

In [17]:
schema_name = 'ed-fi'
ext_entity = 'TPDM'
test_mode = False
transform_mode = True
tables_source = f'{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/{schema_name}'
transform_items = 'All'#['staffs', 'students', 'staffEducationOrganizationAssignmentAssociations']

refine_and_explode_data(schema_name, 
                        tables_source,
                        ext_entity,
                        transform_mode, 
                        test_mode,
                        test_items = transform_items)